<a href="https://colab.research.google.com/github/tcnicholas/topo-metrics/blob/main/notebooks/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 `topo-metrics` quickstart!
`topo-metrics` provides a Python interface to the powerful [`CrystalNets.jl`](https://coudertlab.github.io/CrystalNets.jl/dev/#CrystalNets.jl) and [`PeriodicGraphs.jl`](https://github.com/Liozou/PeriodicGraphs.jl) Julia packages — letting you compute ring statistics and topological metrics for network materials with ease!

### ⚙️ One-time Setup (Don't Skip!)

Before we can run any analysis, we need to set up the Julia—Python bridge using [`PythonCall.jl`](https://juliapy.github.io/PythonCall.jl/stable/). This means:

- Installing Julia packages 📦

- Resolving dependencies 🔧

- Precompiling everything for speed later 🧠

**🐢 Heads up**: This step can take a *long time* on Google Colab (sometimes up to 30 minutes...).
So go ahead — run the cell below and grab a coffee ☕️. We'll be ready to explore topology by the time you're back!

In [1]:
%%capture

# install Julia using official script.
!curl -fsSL https://install.julialang.org | sh -s -- --yes

# add Julia to PATH.
import os
os.environ["PATH"] += ":/root/.juliaup/bin"

# install `topo-metrics`.
!pip install topo-metrics

# we need to explicitly setup the Julia env for collab.
import juliacall
from juliacall import Main as jl

# trigger Julia environment setup.
jl.seval("""
import Pkg
Pkg.activate("/usr/local/lib/python3.11/dist-packages/topo_metrics/RingStatistics")
Pkg.instantiate()
""")

# once more, we will trigger the julia setup within topo-metrics (which runs
# the first the program is called).
import topo_metrics

# finally, let's download some example data.
from pathlib import Path
from urllib.request import urlretrieve

example_dir = Path("examples/data")
example_dir.mkdir(parents=True, exist_ok=True)

url = "https://raw.githubusercontent.com/tcnicholas/topo-metrics/main/examples/data/alpha-cristobalite.cgd"
urlretrieve(url, example_dir / "alpha-cristobalite.cgd")

## ✅ We're ready to go!

---

In [3]:
from topo_metrics import Topology

### 🧪 Example: α-Cristobalite

Our example is based on the **α-cristobalite** structure.  
We’ve already downloaded a Systre Crystallographic Geometry Definition (CGD) file, so let’s load it into `topo-metrics`.

In [4]:
topology = Topology.from_cgd(example_dir / "alpha-cristobalite.cgd")
topology

Topology(nodes=4, edges=16, has_lattice=True)

Out of the box, we can pass the resulting periodic graph to [`CrystalNets.jl`](https://github.com/smtg-bham/CrystalNets.jl) to determine the topology of the network.

Since α-cristobalite adopts a **diamond-like structure**, the expected topology code returned by CrystalNets should be:

In [5]:
topology_code = topology.get_topological_genome()
topology_code

'dia'

### 🔄 Ring and Cluster Extraction

The core functionality is accessed through the `Topology.get_rings()` method. This routine identifies all rings in a structure using [`PeriodicGraphs.jl`](https://github.com/Liozou/PeriodicGraphs.jl) under the hood, and returns them as a collection of `Ring` objects.

By gathering all `Ring` instances surrounding a given node, a `Cluster` object can be constructed — representing the local topological environment of that node.

All of this is bundled into a `RingsResults` instance, which we'll unpack shortly!


In [6]:
rings_results = topology.get_rings(depth=6)
rings_results

RingsResults(
    depth=6,
    strong_rings=False,
    ring_size_count=RingSizeCounts(n_rings=8, min=6, max=6),
    VertexSymbol=[6(2).6(2).6(2).6(2).6(2).6(2)],
    CARVS={6(12)}
)

### 📊 Global Ring Statistics

Let's start with the simplest view: the **global** ring statistics for the structure.  
The `RingSizeCounts` object stores the distribution of ring sizes detected across the entire network.

Below is a summary of the non-zero ring counts:

In [16]:
print("Ring Size | Count")
print("----------|------")
for size, count in rings_results.ring_size_count:
    if count > 0:
        print(f"{size:^9} | {count:^5}")


Ring Size | Count
----------|------
    6     |   8  
